In [1]:
from datasets import load_dataset

In [2]:
corpus = load_dataset('scifact', 'corpus')
claims = load_dataset('scifact', 'claims')


Reusing dataset scifact (/Users/domenicrosati/.cache/huggingface/datasets/scifact/corpus/1.0.0/15660e43ecfb3f7420850027005a63611abb2d401e9746b4059c1260745d9831)


  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset scifact (/Users/domenicrosati/.cache/huggingface/datasets/scifact/claims/1.0.0/15660e43ecfb3f7420850027005a63611abb2d401e9746b4059c1260745d9831)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
corpus

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'title', 'abstract', 'structured'],
        num_rows: 5183
    })
})

In [4]:
claims

DatasetDict({
    train: Dataset({
        features: ['id', 'claim', 'evidence_doc_id', 'evidence_label', 'evidence_sentences', 'cited_doc_ids'],
        num_rows: 1261
    })
    test: Dataset({
        features: ['id', 'claim', 'evidence_doc_id', 'evidence_label', 'evidence_sentences', 'cited_doc_ids'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['id', 'claim', 'evidence_doc_id', 'evidence_label', 'evidence_sentences', 'cited_doc_ids'],
        num_rows: 450
    })
})

In [5]:
df = claims['train'].to_pandas()

In [6]:
corpus_df = corpus['train'].to_pandas()

In [7]:
new_df = df.loc[(df['evidence_label'] == 'CONTRADICT') | (df['evidence_label'] == 'SUPPORT')]

In [8]:
corpus_df['doc_id_str'] = corpus_df['doc_id'].apply(lambda x: str(x))

In [9]:
for i, doc in new_df.iterrows():
    claim = doc['claim']
    evidence = corpus_df[corpus_df['doc_id_str'].apply(lambda x: str(x)) == doc['evidence_doc_id']]['abstract'].iloc[0][doc['evidence_sentences'][0]]
    type_ = doc['evidence_label']
    score = 1 if type_ == 'CONTRADICT' else 0

In [10]:
print(len(new_df))

957


In [11]:
wiki_pages = load_dataset('fever', 'wiki_pages')

Reusing dataset fever (/Users/domenicrosati/.cache/huggingface/datasets/fever/wiki_pages/1.0.0/366810bbb7a3fa7bdc828fdac879cd1bef00c99b4fe60ba513ab38b1668bf5f0)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
fever = load_dataset('fever', 'v1.0')

Reusing dataset fever (/Users/domenicrosati/.cache/huggingface/datasets/fever/v1.0/1.0.0/366810bbb7a3fa7bdc828fdac879cd1bef00c99b4fe60ba513ab38b1668bf5f0)


  0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
fever['train'][0]

{'id': 75397,
 'label': 'SUPPORTS',
 'claim': 'Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.',
 'evidence_annotation_id': 92206,
 'evidence_id': 104971,
 'evidence_wiki_url': 'Nikolaj_Coster-Waldau',
 'evidence_sentence_id': 7}

In [14]:
'Nikolaj_Coster-Waldau' in wiki_pages['wikipedia_pages']['id']

True

In [15]:
fever_df = fever['train'].to_pandas()

In [16]:
wiki_df = wiki_pages['wikipedia_pages'].to_pandas()

In [17]:
wiki_df[wiki_df['id'] == 'Nikolaj_Coster-Waldau']['lines'].iloc[0].split('\n')[7].split('\t')[1]

'He then played Detective John Amsterdam in the short-lived Fox television series New Amsterdam -LRB- 2008 -RRB- , as well as appearing as Frank Pike in the 2009 Fox television film Virtuality , originally intended as a pilot .'

In [18]:
fever_filtered = fever_df[fever_df['evidence_sentence_id'] != -1]
fever_filtered = fever_filtered[(fever_filtered['label'] == 'SUPPORTS') | (fever_filtered['label'] == 'REFUTES')]

In [19]:
len(fever_filtered)

168293

In [20]:
from tqdm import tqdm

In [24]:
wiki_pages = load_dataset('fever', 'wiki_pages')
wiki_df = wiki_pages['wikipedia_pages'].to_pandas()
fever = load_dataset('fever', 'v1.0')

EVIDENCE_LABEL = 'SUPPORTS'

fever_df = fever['train'].to_pandas()
fever_filtered = fever_df[fever_df['evidence_sentence_id'] != -1]
fever_filtered = fever_filtered[(fever_filtered['label'] == 'SUPPORTS') | (fever_filtered['label'] == 'REFUTES')]

wiki_df_filtered = wiki_df[wiki_df['id'].isin(fever_filtered['evidence_wiki_url'])]



Reusing dataset fever (/Users/domenicrosati/.cache/huggingface/datasets/fever/wiki_pages/1.0.0/366810bbb7a3fa7bdc828fdac879cd1bef00c99b4fe60ba513ab38b1668bf5f0)


  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset fever (/Users/domenicrosati/.cache/huggingface/datasets/fever/v1.0/1.0.0/366810bbb7a3fa7bdc828fdac879cd1bef00c99b4fe60ba513ab38b1668bf5f0)


  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:

train_samples = []
train_samples_ranking = {}
train_df = fever_filtered 
for i, doc in tqdm(train_df.iterrows(), total=len(train_df)):
    try:
        claim = doc['claim']
        wiki_sentences = wiki_df_filtered[wiki_df_filtered['id'] == doc['evidence_wiki_url']]['lines'].iloc[0].split('\n')
        evidence = wiki_sentences[
            doc['evidence_sentence_id']
        ].split('\t')[1]
        type_ = doc['label']
        score = 1 if type_ == EVIDENCE_LABEL else 0
        train_samples.append([claim, evidence, score])
        
        if claim not in train_samples_ranking:
            train_samples_ranking[claim] = {
                'query': claim, 'positive': [], 'negative': []
            }
        
        if score == 1:
            train_samples_ranking[claim]['positive'].append(evidence)
        
        train_samples_ranking[claim]['negative'].extend(
            sent.split('\t')[1] for sent in
            wiki_df_filtered[wiki_df_filtered['id'] == doc['evidence_wiki_url']]['lines'].iloc[0].split('\n')
            if sent.split('\t')[1] != evidence and score == 1
        )
        
            
        
    except Exception as e:
        pass
    # train_samples.append(InputExample(texts=[claim, evidence], label=score))

 24%|██▎       | 39944/168293 [01:05<03:39, 584.47it/s]

In [ ]:
len(train_samples_ranking)

In [ ]:
ranking_samples_filtered = []
for sample in list(train_samples_ranking.values()):
    if len(sample['positive']) == 0 or len(sample['negative']) == 0:
        continue
    ranking_samples_filtered.append(sample)

In [ ]:
len(ranking_samples_filtered)

In [ ]:
ranking_samples_filtered[0]